## 주소 기준으로 패널 생성 및 회귀분석 실행하기

### To do
- 위도, 경도를 concatenate해서 주소 고유값을 분별할 수 있게 하는 컬럼 생성(latitude + longitude)
- 주소(고유 ID)를 기준으로 panel 자료 만들기
- house_type에 따라 csv 파일 생성하기

In [56]:
import numpy as np
import pandas as pd

# 데이터 불러오기
df = pd.read_csv('after_EDA1_non_outlier_region_add2.csv')
df.head()

,Unnamed: 0,index,property_U,listed_dat,reupload,new,descrip10,descriptio,Floorsize,Floor,...,image_urls,URL,developmen,properties,OBJECTID,PLN_AREA_N,PLN_AREA_C,CA_IND,REGION_N,REGION_C
0,0,100,Middle Floor1.411856103.831219excellent unit s...,2019-10-26,new,1,excellent unit sale miss 2 bed rom sale go fast,excellent unit sale miss 2 bed rom sale go fas...,930,Middle Floor,...,https://sg1-cdn.pgimgs.com/listing/22073076/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
1,1,105,Middle Floor1.41275103.835467great deal miss r...,2019-10-26,new,1,great deal miss rarely avail 3 bed sale go fast,great deal miss rarely avail 3 bed sale go fas...,1143,Middle Floor,...,https://sg1-cdn.pgimgs.com/listing/22061998/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
2,2,446,High Floor1.4239069103.8482042canopy exe condo...,2019-10-26,new,1,canopy exe condo available single spr 21 years...,canopy exe condo available single spr 21 years...,1033,High Floor,...,https://sg2-cdn.pgimgs.com/listing/22036970/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
3,3,568,High Floor1.438264103.830611big 2 bedrooms sal...,2019-10-26,new,1,big 2 bedrooms sale sale want stay high floor ...,big 2 bedrooms sale sale want stay high floor ...,1001,High Floor,...,https://sg2-cdn.pgimgs.com/listing/21225562/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/project/yishun-eme...,25,YISHUN,YS,N,NORTH REGION,NR
4,4,599,High Floor1.41275103.8354673br awesome full re...,2019-10-26,new,1,3br awesome full reservoir view rare full unbl...,3br awesome full reservoir view rare full unbl...,1248,High Floor,...,https://sg2-cdn.pgimgs.com/listing/21862086/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/project/orchid-par...,25,YISHUN,YS,N,NORTH REGION,NR


In [57]:
# 컬럼명 확인
df.columns

Index(['Unnamed: 0', 'index', 'property_U', 'listed_dat', 'reupload', 'new',
       'descrip10', 'descriptio', 'Floorsize', 'Floor', 'House_type',
       'Lease_type', 'region', 'ListingID', 'created_at', 'years_from',
       'PSF_float', 'Tenanted', 'Total_Unit', 'Furnishing', 'listed_yea',
       'Month', 'phase', 'pandemic', 'address_lo', 'latitude', 'longitude',
       'rooms', 'bathrooms', 'floor_size', 'price', 'days_past', 'Condo',
       'HDB', 'TOP', 'Lease', 'Free', 'Developer', 'image_urls', 'URL',
       'developmen', 'properties', 'OBJECTID', 'PLN_AREA_N', 'PLN_AREA_C',
       'CA_IND', 'REGION_N', 'REGION_C'],
      dtype='object')

In [58]:
# 고유한 주소 ID 생성
df['address_ID'] = df['latitude'].astype(str) + '_' + df['longitude'].astype(str)
df[['latitude', 'longitude', 'address_ID']].head(6)

,latitude,longitude,address_ID
0,1.411856,103.831219,1.411856_103.83121899999999
1,1.412750,103.835467,1.41275_103.83546700000001
2,1.423907,103.848204,1.4239069_103.8482042
3,1.438264,103.830611,1.438264_103.830611
4,1.412750,103.835467,1.41275_103.83546700000001
5,1.439143,103.831766,1.439143464_103.8317657


In [59]:
from category_encoders import OrdinalEncoder

# ordianl encorder를 통해 address_ID가 동일하는 같은 ID로 만들어줌
encorder = OrdinalEncoder(df['address_ID'])
lst = encorder.fit_transform(df['address_ID'])
df['address_ID'] = lst

In [60]:
df[['latitude', 'longitude', 'address_ID']].head(10)

,latitude,longitude,address_ID
0,1.411856,103.831219,1
1,1.412750,103.835467,2
2,1.423907,103.848204,3
3,1.438264,103.830611,4
4,1.412750,103.835467,2
5,1.439143,103.831766,5
6,1.412750,103.835467,2
7,1.424436,103.845123,6
8,1.417853,103.847470,7
9,1.439143,103.831766,5


In [61]:
# 컬럼 위치 수정 및 불필요한 unamed_0, index 삭제
df = df.reindex(columns = ['property_ID', 'address_ID', 'listed_dat', 'reupload',
                      'new', 'descrip10', 'descriptio', 'Floorsize', 'Floor', 'House_type', 
                      'Lease_type', 'region', 'ListingID', 'created_at', 'years_from', 
                      'PSF_float', 'Tenanted','Total_Unit','Furnishing','listed_yea', 
                      'Month', 'phase', 'pandemic', 'address_lo', 'latitude', 'longitude', 
                      'rooms', 'bathrooms', 'floor_size', 'price', 'days_past', 'Condo','HDB',
                      'TOP', 'Lease', 'Free', 'Developer', 'image_urls', 'URL', 'developmen', 
                      'properties', 'OBJECTID', 'PLN_AREA_N', 'PLN_AREA_C', 'CA_IND', 
                      'REGION_N', 'REGION_C'])

# 컬럼명 변경
df.rename(columns = {'property_U' : 'property_UID', 'listed_dat':'listed_date', 'descriptio' : 'description','years_from':'years_from_TOP','listed_yea': 'listed_year',
                    'developmen': 'development_properties.Completion Year','properties': 'properties.url'}, inplace = True)
df.head()

,property_ID,property_UID,address_ID,listed_date,reupload,new,descrip10,description,Floorsize,Floor,...,image_urls,URL,development_properties.Completion Year,properties.url,OBJECTID,PLN_AREA_N,PLN_AREA_C,CA_IND,REGION_N,REGION_C
0,NaN,Middle Floor1.411856103.831219excellent unit s...,1,2019-10-26,new,1,excellent unit sale miss 2 bed rom sale go fast,excellent unit sale miss 2 bed rom sale go fas...,930,Middle Floor,...,https://sg1-cdn.pgimgs.com/listing/22073076/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
1,NaN,Middle Floor1.41275103.835467great deal miss r...,2,2019-10-26,new,1,great deal miss rarely avail 3 bed sale go fast,great deal miss rarely avail 3 bed sale go fas...,1143,Middle Floor,...,https://sg1-cdn.pgimgs.com/listing/22061998/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
2,NaN,High Floor1.4239069103.8482042canopy exe condo...,3,2019-10-26,new,1,canopy exe condo available single spr 21 years...,canopy exe condo available single spr 21 years...,1033,High Floor,...,https://sg2-cdn.pgimgs.com/listing/22036970/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
3,NaN,High Floor1.438264103.830611big 2 bedrooms sal...,4,2019-10-26,new,1,big 2 bedrooms sale sale want stay high floor ...,big 2 bedrooms sale sale want stay high floor ...,1001,High Floor,...,https://sg2-cdn.pgimgs.com/listing/21225562/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/project/yishun-eme...,25,YISHUN,YS,N,NORTH REGION,NR
4,NaN,High Floor1.41275103.8354673br awesome full re...,2,2019-10-26,new,1,3br awesome full reservoir view rare full unbl...,3br awesome full reservoir view rare full unbl...,1248,High Floor,...,https://sg2-cdn.pgimgs.com/listing/21862086/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/project/orchid-par...,25,YISHUN,YS,N,NORTH REGION,NR


In [62]:
# 주택 유형이 '콘도'인 것만 추출
df_condo = df.loc[df['House_type'] == 'Condo'].reset_index(drop = True)
print('Condo의 데이터 갯수 :', df_condo.shape[0])
df_condo.head()

Condo의 데이터 갯수 : 131146


,property_ID,property_UID,address_ID,listed_date,reupload,new,descrip10,description,Floorsize,Floor,...,image_urls,URL,development_properties.Completion Year,properties.url,OBJECTID,PLN_AREA_N,PLN_AREA_C,CA_IND,REGION_N,REGION_C
0,NaN,Middle Floor1.411856103.831219excellent unit s...,1,2019-10-26,new,1,excellent unit sale miss 2 bed rom sale go fast,excellent unit sale miss 2 bed rom sale go fas...,930,Middle Floor,...,https://sg1-cdn.pgimgs.com/listing/22073076/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
1,NaN,Middle Floor1.41275103.835467great deal miss r...,2,2019-10-26,new,1,great deal miss rarely avail 3 bed sale go fast,great deal miss rarely avail 3 bed sale go fas...,1143,Middle Floor,...,https://sg1-cdn.pgimgs.com/listing/22061998/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
2,NaN,High Floor1.4239069103.8482042canopy exe condo...,3,2019-10-26,new,1,canopy exe condo available single spr 21 years...,canopy exe condo available single spr 21 years...,1033,High Floor,...,https://sg2-cdn.pgimgs.com/listing/22036970/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
3,NaN,High Floor1.438264103.830611big 2 bedrooms sal...,4,2019-10-26,new,1,big 2 bedrooms sale sale want stay high floor ...,big 2 bedrooms sale sale want stay high floor ...,1001,High Floor,...,https://sg2-cdn.pgimgs.com/listing/21225562/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/project/yishun-eme...,25,YISHUN,YS,N,NORTH REGION,NR
4,NaN,High Floor1.41275103.8354673br awesome full re...,2,2019-10-26,new,1,3br awesome full reservoir view rare full unbl...,3br awesome full reservoir view rare full unbl...,1248,High Floor,...,https://sg2-cdn.pgimgs.com/listing/21862086/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/project/orchid-par...,25,YISHUN,YS,N,NORTH REGION,NR


In [63]:
# 주택 유형이 'HDB'인 것만 추출
df_HDB = df.loc[df['House_type'] == 'HDB'].reset_index(drop = True)
print('HDB의 데이터 갯수 :', df_HDB.shape[0])
df_HDB.head()

HDB의 데이터 갯수 : 76015


,property_ID,property_UID,address_ID,listed_date,reupload,new,descrip10,description,Floorsize,Floor,...,image_urls,URL,development_properties.Completion Year,properties.url,OBJECTID,PLN_AREA_N,PLN_AREA_C,CA_IND,REGION_N,REGION_C
0,NaN,missing1.419751126103.8405439corner renovate j...,62,2019-10-26,new,1,corner renovate jumbo nice blk 665 jumbo multi...,corner renovate jumbo nice blk 665 jumbo multi...,1765,missing,...,https://sg2-cdn.pgimgs.com/listing/21894619/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/singapore-property...,25,YISHUN,YS,N,NORTH REGION,NR
1,NaN,Middle Floor1.415942522103.8368744new list 3 r...,63,2019-10-26,new,1,new list 3 room unit sale grab 3a unit sale,new list 3 room unit sale grab 3a unit sale ex...,796,Middle Floor,...,https://sg2-cdn.pgimgs.com/listing/22058843/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
2,NaN,Low Floor1.422951103.8468481renovate 5i sale y...,64,2019-10-26,reupload,0,renovate 5i sale yishun ave 11 serious motivat...,renovate 5i sale yishun ave 11 serious motivat...,1206,Low Floor,...,https://sg2-cdn.pgimgs.com/listing/22068978/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
3,NaN,Middle Floor1.4269717103.8422562adora green 5 ...,65,2019-10-26,new,1,adora green 5 years old newest hdb yishun show...,adora green 5 years old newest hdb yishun show...,990,Middle Floor,...,https://sg2-cdn.pgimgs.com/listing/21997616/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/singapore-property...,25,YISHUN,YS,N,NORTH REGION,NR
4,NaN,Middle Floor1.4319674999999998103.842574200000...,66,2019-10-26,new,1,sale 4 bedroom near junction 9 5i yishun adi m...,sale 4 bedroom near junction 9 5i yishun adi m...,1206,Middle Floor,...,https://sg2-cdn.pgimgs.com/listing/22065872/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/singapore-property...,25,YISHUN,YS,N,NORTH REGION,NR


In [64]:
# 데이터 프레임 csv로 만들기
df.to_csv('정호섭 전처리 파트 (모든 주택 유형).csv', index = False)
df_condo.to_csv('정호섭 전처리 파트 (콘도).csv', index = False)
df_HDB.to_csv('정호섭 전처리 파트 (HDB).csv', index = False)

In [69]:
df = pd.read_csv('정호섭 전처리 파트 (모든 주택 유형).csv')
print(df.shape)
df.head()

(239974, 48)


,property_ID,property_UID,address_ID,listed_date,reupload,new,descrip10,description,Floorsize,Floor,...,image_urls,URL,development_properties.Completion Year,properties.url,OBJECTID,PLN_AREA_N,PLN_AREA_C,CA_IND,REGION_N,REGION_C
0,NaN,Middle Floor1.411856103.831219excellent unit s...,1,2019-10-26,new,1,excellent unit sale miss 2 bed rom sale go fast,excellent unit sale miss 2 bed rom sale go fas...,930,Middle Floor,...,https://sg1-cdn.pgimgs.com/listing/22073076/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
1,NaN,Middle Floor1.41275103.835467great deal miss r...,2,2019-10-26,new,1,great deal miss rarely avail 3 bed sale go fast,great deal miss rarely avail 3 bed sale go fas...,1143,Middle Floor,...,https://sg1-cdn.pgimgs.com/listing/22061998/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
2,NaN,High Floor1.4239069103.8482042canopy exe condo...,3,2019-10-26,new,1,canopy exe condo available single spr 21 years...,canopy exe condo available single spr 21 years...,1033,High Floor,...,https://sg2-cdn.pgimgs.com/listing/22036970/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/,25,YISHUN,YS,N,NORTH REGION,NR
3,NaN,High Floor1.438264103.830611big 2 bedrooms sal...,4,2019-10-26,new,1,big 2 bedrooms sale sale want stay high floor ...,big 2 bedrooms sale sale want stay high floor ...,1001,High Floor,...,https://sg2-cdn.pgimgs.com/listing/21225562/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/project/yishun-eme...,25,YISHUN,YS,N,NORTH REGION,NR
4,NaN,High Floor1.41275103.8354673br awesome full re...,2,2019-10-26,new,1,3br awesome full reservoir view rare full unbl...,3br awesome full reservoir view rare full unbl...,1248,High Floor,...,https://sg2-cdn.pgimgs.com/listing/21862086/UP...,http://api.scraperapi.com/?api_key=b0560c3ca5b...,NaN,https://propertyguru.com.sg/project/orchid-par...,25,YISHUN,YS,N,NORTH REGION,NR
